In [ ]:
# setup
# !pip install -U pyTigerGraph

# Imports
import pyTigerGraph as tg
import json
import pandas as pd

# Connection parameters

# TODO
hostName = "https://gsql-101.i.tgcloud.io"
userName = "user_1"
password = "Tz4Do8Ec4Ht0Ed2+"

conn = tg.TigerGraphConnection(host=hostName, username=userName, password=password, graphname="MyGraph")

print("Connected") 

### Define and Publish Graph Schema

Nodes => ETH Users/wallet addresses: ID, label <br />
Directed Edges => Transaction flow: From_ID, To_ID, ETH_amount, timestamp

In [ ]:
# DEFINE / CREATE ALL EDGES AND VERTICES 
results = conn.gsql('''
  USE GLOBAL
  CREATE VERTEX Wallet (PRIMARY_ID id STRING, label STRING) WITH primary_id_as_attribute="true"
  CREATE DIRECTED EDGE sent_eth (From Wallet, To Wallet, Amount INT, sent_date DATETIME) WITH REVERSE_EDGE="reverse_sent_eth"
  CREATE DIRECTED EDGE received_eth (From Wallet, To Wallet, Amount INT, receive_date DATETIME) WITH REVERSE_EDGE="reverse_received_eth"
''')
print(results)

### Create the Ethereum Transaction Graph

In [ ]:
results = conn.gsql('''
  CREATE GRAPH MyGraph(Wallet, sent_eth, reverse_sent_eth, received_eth, reverse_received_eth)
''')
print(results)

In [ ]:
conn.graphname="ETH_Transaction_Graph"
secret = conn.createSecret()
authToken = conn.getToken(secret)
authToken = authToken[0]
print(authToken)
# authToken = 'rc7reopbis1667ksgcppq5v5fb99p6s1'
conn = tg.TigerGraphConnection(host=hostName, graphname="ETH_Transaction_Graph", username=userName, password=password, apiToken=authToken)

def pprint(string):
  print(json.dumps(string, indent=2))

### Create Loading Jobs

#### Wallets

In [ ]:
#TODO - change data/nodes.txt to nodes.csv?

results = conn.gsql('''
  USE GRAPH MyGraph
  BEGIN
  CREATE LOADING JOB load_wallets FOR GRAPH MyGraph {
  DEFINE FILENAME MyDataSource;
  LOAD MyDataSource TO VERTEX Wallet VALUES($0, $1) USING SEPARATOR=",", HEADER="true", EOL="\\n", QUOTE="double";
  }
  END
  ''')
print(results)

#### Transactions

In [ ]:
# TODO - change data/edges.txt to edges.csv?

results = conn.gsql('''
  USE GRAPH MyGraph
  BEGIN
  CREATE LOADING JOB load_transactions FOR GRAPH MyGraph {
  DEFINE FILENAME MyDataSource;
  LOAD MyDataSource TO EDGE sent_eth VALUES($3, $0, $4, $5) USING SEPARATOR=",", HEADER="true", EOL="\\n", QUOTE="double";
  LOAD MyDataSource TO EDGE received_eth VALUES($0, $4, $3, $5, $6) USING SEPARATOR=",", HEADER="true", EOL="\\n", QUOTE="double";
  }
  END''')
print(results)

### Load Data

In [ ]:
#TODO

# Load the nodes file wiht the 'load_wallets' job
posts_file = '/data/nodes.csv'
results = conn.uploadFile(posts_file, fileTag='MyDataSource', jobName='load_wallets')
print(json.dumps(results, indent=2))

In [ ]:
# TODO

# Load the edges file wiht the 'load_transactions' job
posts_file = '/data/edges.csv'
results = conn.uploadFile(posts_file, fileTag='MyDataSource', jobName='load_transactions')
print(json.dumps(results, indent=2))

### Exploring the Graph

In [ ]:
# TODO - ETHAN

# write GSQL queries to get some summary statistics on edges/nodes
# use this as reference: https://colab.research.google.com/drive/1JhYcnGVWT51KswcXZzyPzKqCoPP5htcC?usp=sharing#scrollTo=Zerob6pDgmq2 

### Clear Graph

# conn.gsql('''
# USE GLOBAL
# DROP ALL
# ''')